In [3]:
import requests
from urllib.parse import urlparse, parse_qs
import pandas as pd
import math

C:\Users\henri\AppData\Local\Temp\ipykernel_15204\3001212684.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [8]:
def get_most_viewed(yt_link):

    # Parse the URL
    parsed_url = urlparse(yt_link)

    # Extract the video ID from the query parameters
    video_id = parse_qs(parsed_url.query).get("v", [])[0] if "v" in parse_qs(parsed_url.query) else None

    if not video_id:
        print("Unable to extract video ID from the URL.")
   
    api_url = "https://yt.lemnoslife.com/videos"
    params = {
        "part": "mostReplayed",
        "id": video_id
    }

    response = requests.get(api_url, params=params)

    if response.status_code == 200:
        # The request was successful
        data = response.json()
        print(data)
        markers = data["items"][0]["mostReplayed"]["markers"]
        df = pd.DataFrame(markers)
        df = df.iloc[1:-1]
        #most_viewed = [element for element in markers if element["intensityScoreNormalized"] > 0.8]
        pas = df["startMillis"][1]
        rows_3min = math.ceil(180000/pas)
        df['sum'] = df.apply(lambda row: df['intensityScoreNormalized'][row.name:row.name+rows_3min+1].sum(), axis=1)
        max_score_index = df['sum'].idxmax()
        best_start = df.loc[max_score_index, 'startMillis']
        return best_start, round((pas*rows_3min)/1000)
    else:
        # The request failed
        print(f"Error: {response.status_code} - {response.text}")


In [10]:

get_most_viewed("https://www.youtube.com/watch?v=5wQvdPtyuKs")

{'error': {'code': 403, 'message': 'YouTube has detected unusual traffic from this YouTube operational API instance. Please try your request again later or see alternatives at https://github.com/Benjamin-Loison/YouTube-operational-API/issues/11'}}


KeyError: 'items'